<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [1]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [2]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

OperationalError: no such table: houses

In [3]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [4]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale)
# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [5]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
# 
c.executemany('INSERT INTO houses VALUES(?,?,?,?,?)', recent_sales)
conn.commit()

Select all rows from houses

In [6]:
# A
c.execute('SELECT * FROM houses').fetchall()



[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [7]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [8]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [9]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [10]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [11]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [12]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do?

All records in the table will be deleted

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [13]:
# ANSWER
c.execute('SELECT * FROM houses WHERE bdrms = 4').fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [14]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()



[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [15]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [16]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('housing-data.csv', low_memory = False)
data.head()

sqft  bdrms  age   price
0  2104      3   70  399900
1  1600      3   28  329900
2  2400      3   44  369000
3  1416      2   49  232000
4  3000      4   75  539900

We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [17]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [21]:
#ANSWER
result = pd.read_sql_query('SELECT bdrms, AVG(price) as average_price FROM houses_pandas GROUP BY bdrms', conn)

result.head()

bdrms  average_price
0      1  169900.000000
1      2  280866.666667
2      3  326403.920000
3      4  377449.785714
4      5  699900.000000

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

1) When the data is unstructured and you want the database to store data in a tabular format

2) When the dataset retreivewed is unstructured and you want to query from the data in a tabular format 

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [22]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [23]:
# Reading CSV to Dataframe
orders = pd.read_csv('P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('P12-OrderBreakdown.csv', encoding = 'utf-8')

In [42]:
orders.head()


order_id  order_date      customer_name         country
0  IT-2011-3647632  2011-01-01       Eugene Moren          Sweden
1  ES-2011-4869686  2011-01-03  Dorothy Dickinson  United Kingdom
2  ES-2011-4939443  2011-01-04     Arthur Prichep          France
3  IT-2011-2942451  2011-01-04     Grant Thornton  United Kingdom
4  ES-2011-3848439  2011-01-05   Michael Granlund          France

In [43]:
orders_break_down.head()


order_id                           product_name  discount  sales  \
0  IT-2011-3647632            Enermax Note Cards, Premium       0.5   45.0   
1  ES-2011-4869686     Dania Corner Shelving, Traditional       0.0  854.0   
2  ES-2011-4939443  Binney & Smith Sketch Pad, Easy-Erase       0.0  140.0   
3  IT-2011-2942451             Boston Markers, Easy-Erase       0.5   27.0   
4  IT-2011-2942451            Eldon Folders, Single Width       0.5   17.0   

   quantity         category  
0         3  Office Supplies  
1         7        Furniture  
2         3  Office Supplies  
3         2  Office Supplies  
4         2  Office Supplies

#### 2. Replace Space with Underscore in Column Names

In [26]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [27]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes 

In [28]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [29]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [53]:
# Establishing Local DB connection
db_connection = sqlite3.connect('eshop.db.sqlite')

# Save these two dataframes as a table in sqlite
orders.to_sql('orders', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql('orders_breakdown', con = db_connection, if_exists = 'replace', index = False)

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [54]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT * 
        FROM orders 
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

order_id  order_date      customer_name         country
0  IT-2011-3647632  2011-01-01       Eugene Moren          Sweden
1  ES-2011-4869686  2011-01-03  Dorothy Dickinson  United Kingdom
2  ES-2011-4939443  2011-01-04     Arthur Prichep          France
3  IT-2011-2942451  2011-01-04     Grant Thornton  United Kingdom
4  ES-2011-3848439  2011-01-05   Michael Granlund          France
5  ES-2011-5433855  2011-01-07       Dave Poirier          France
6  IT-2011-4546695  2011-01-08      Darren Powers          France
7  ES-2011-1138719  2011-01-11       Eric Murdock           Italy
8  ES-2011-1466305  2011-01-11         Mick Brown         Austria
9  ES-2011-4359424  2011-01-11  Dorothy Dickinson           Spain

In [55]:
# Select Number of Orders for Each Customer
query1 = """
         SELECT customer_name, COUNT(DISTINCT order_id)
         FROM orders
         GROUP BY customer_name;
         """
no_of_orders = sql.read_sql(query1, con=db_connection)
no_of_orders

customer_name  COUNT(DISTINCT order_id)
0         Aaron Bergman                        11
1         Aaron Hawkins                         7
2        Aaron Smayling                        12
3       Adam Bellavance                         4
4             Adam Hart                         8
..                  ...                       ...
787        Xylona Preis                         2
788       Yana Sorensen                         5
789      Yoseph Carroll                        11
790    Zuschuss Carroll                         9
791  Zuschuss Donatelli                         6

[792 rows x 2 columns]

#### 6. Select Number of Customers for Each Country

In [57]:
# Select Number of Customers for Each Country

query3 = """
         SELECT country,COUNT(DISTINCT customer_name)
         FROM orders
         GROUP BY country;
         """
customer_country = sql.read_sql(query3, con = db_connection)
customer_country

country  COUNT(DISTINCT customer_name)
0          Austria                            125
1          Belgium                             66
2          Denmark                             28
3          Finland                             33
4           France                            568
5          Germany                            505
6          Ireland                             48
7            Italy                            351
8      Netherlands                            172
9           Norway                             37
10        Portugal                             36
11           Spain                            306
12          Sweden                             96
13     Switzerland                             39
14  United Kingdom                            473

#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [59]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table
query4 = """
         SELECT order_id, sales, quantity, discount
         FROM orders_breakdown
         GROUP BY order_id;
         """

query_4 = sql.read_sql(query4, con = db_connection)
query_4

order_id   sales  quantity  discount
0     ES-2011-1001989   229.0         9       0.1
1     ES-2011-1010958    58.0         2       0.0
2     ES-2011-1012469   148.0         3       0.0
3     ES-2011-1043483   224.0        11       0.0
4     ES-2011-1058269    13.0         2       0.0
...               ...     ...       ...       ...
4112  IT-2014-5966070   107.0         4       0.5
4113  IT-2014-5975833   656.0         2       0.1
4114  IT-2014-5984498    83.0         1       0.5
4115  IT-2014-5989338  1603.0         9       0.1
4116  IT-2014-5992832    74.0         8       0.0

[4117 rows x 4 columns]

#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [60]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table
query5 = """
         SELECT order_id, sales, quantity, discount, (sales*quantity - discount) AS total_price
         FROM orders_breakdown
         GROUP BY order_id;
         """

query_5 = sql.read_sql(query5, con = db_connection)
query_5

order_id   sales  quantity  discount  total_price
0     ES-2011-1001989   229.0         9       0.1       2060.9
1     ES-2011-1010958    58.0         2       0.0        116.0
2     ES-2011-1012469   148.0         3       0.0        444.0
3     ES-2011-1043483   224.0        11       0.0       2464.0
4     ES-2011-1058269    13.0         2       0.0         26.0
...               ...     ...       ...       ...          ...
4112  IT-2014-5966070   107.0         4       0.5        427.5
4113  IT-2014-5975833   656.0         2       0.1       1311.9
4114  IT-2014-5984498    83.0         1       0.5         82.5
4115  IT-2014-5989338  1603.0         9       0.1      14426.9
4116  IT-2014-5992832    74.0         8       0.0        592.0

[4117 rows x 5 columns]

#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [64]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100
query6 = """
         SELECT order_id,(sales*quantity - discount) AS total_price
         FROM orders_breakdown
         WHERE total_price > 100
         """
price_100 = sql.read_sql(query6, con = db_connection)
price_100

order_id  total_price
0     IT-2011-3647632        134.5
1     ES-2011-4869686       5978.0
2     ES-2011-4939443        420.0
3     ES-2011-3848439        270.0
4     ES-2011-3848439        828.0
...               ...          ...
6461  ES-2014-2815584       1566.0
6462  ES-2014-4785777        160.0
6463  ES-2014-3458802        490.0
6464  IT-2014-3715679        323.5
6465  ES-2014-3458802       4335.0

[6466 rows x 2 columns]

#### 8. Select All Customer And The Product They Have Bought

In [65]:
# Select All Customer And The Product They Have Bought
query7 = """
         SELECT o.customer_name, b.product_name
         FROM orders o
         JOIN orders_breakdown b
         ON o.order_id = b.order_id
         GROUP BY o.customer_name;
         """

customer_product = sql.read_sql(query7, con = db_connection)
customer_product

customer_name                               product_name
0         Aaron Bergman           Apple Office Telephone, Cordless
1         Aaron Hawkins                     Epson Inkjet, Wireless
2        Aaron Smayling           Binney & Smith Pens, Fluorescent
3       Adam Bellavance            Sauder Classic Bookcase, Mobile
4             Adam Hart            Cardinal Binder Covers, Economy
..                  ...                                        ...
787        Xylona Preis              Ikea Classic Bookcase, Mobile
788       Yana Sorensen     Binney & Smith Sketch Pad, Water Color
789      Yoseph Carroll           Hon Round Labels, 5000 Label Set
790    Zuschuss Carroll                           BIC Canvas, Blue
791  Zuschuss Donatelli  Barricks Round Table, with Bottom Storage

[792 rows x 2 columns]

#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [67]:
# Select Number of 'Furniture' Orders For Each Country
query8 = """
         SELECT o.country, COUNT(b.category) AS furniture_orders
         FROM orders o
         JOIN orders_breakdown b
         ON o.order_id = b.order_id
         WHERE b.category = 'Furniture'
         GROUP BY o.country
         """
furniture_orders = sql.read_sql(query8,con = db_connection)
furniture_orders

country  furniture_orders
0          Austria                40
1          Belgium                20
2          Denmark                 9
3          Finland                16
4           France               299
5          Germany               264
6          Ireland                12
7            Italy               136
8      Netherlands                76
9           Norway                 6
10        Portugal                10
11           Spain               123
12          Sweden                36
13     Switzerland                11
14  United Kingdom               180

#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [68]:
# Select Number of 'Furniture' Orders For The Country Denmark

query9 = """
         SELECT o.country, COUNT(b.category) AS furniture_orders
         FROM orders o
         JOIN orders_breakdown b
         ON o.order_id = b.order_id
         WHERE b.category = 'Furniture'
         AND o.country = 'Denmark'
         GROUP BY o.country
         """
furniture_denmark = sql.read_sql(query9,con = db_connection)
furniture_denmark

country  furniture_orders
0  Denmark                 9

#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [73]:
# Select Total Sales With Discount and Without Discount for Each Country
query10 = """
          SELECT o.country,
          SUM(b.sales*b.quantity)AS total_sales_no_discount,
          SUM(b.sales*b.quantity -b.discount) AS total_sales_with_discount
          FROM orders o
          JOIN orders_breakdown b
          ON o.order_id = b.order_id
          GROUP BY o.country
          """
sale_discount = sql.read_sql(query10, con = db_connection)
sale_discount

country  total_sales_no_discount  total_sales_with_discount
0          Austria                 391694.0                  391694.00
1          Belgium                 229829.0                  229829.00
2          Denmark                  34617.0                   34586.50
3          Finland                  82001.0                   82001.00
4           France                3108592.0                 3108445.00
5          Germany                2411483.0                 2411393.15
6          Ireland                  75724.0                   75673.70
7            Italy                1177921.0                 1177798.40
8      Netherlands                 379714.0                  379525.10
9           Norway                  99878.0                   99878.00
10        Portugal                  94629.0                   94594.00
11           Spain                1340832.0                 1340805.90
12          Sweden                 168461.0                  168357.90
13     Switzerland                 141661.0                  141661.00
14  United Kingdom                2161947.0                 2161854.80

#### 11.A Select Total Quantity, Total Sales for Each Country

In [76]:
# Select Total Quantity, Total Sales for Each Country

query11 = """
          SELECT o.country,
          SUM(b.quantity) as total_quantity,
          SUM(b.sales*b.quantity -b.discount) AS total_sales
          FROM orders o
          JOIN orders_breakdown b
          ON o.order_id = b.order_id
          GROUP BY o.country
          """
country_sales = sql.read_sql(query11, con = db_connection)
country_sales

country  total_quantity  total_sales
0          Austria             973    391694.00
1          Belgium             532    229829.00
2          Denmark             204     34586.50
3          Finland             201     82001.00
4           France            7329   3108445.00
5          Germany            6179   2411393.15
6          Ireland             392     75673.70
7            Italy            3612   1177798.40
8      Netherlands            1526    379525.10
9           Norway             261     99878.00
10        Portugal             286     94594.00
11           Spain            2881   1340805.90
12          Sweden             753    168357.90
13     Switzerland             308    141661.00
14  United Kingdom            4917   2161854.80

#### 11.B Select Top 3 Country Based on Sales

In [78]:
# Select Top 3 Country Based on Sales

query12 = """
          SELECT o.country,
          SUM(b.sales*b.quantity -b.discount) AS total_sales
          FROM orders o
          JOIN orders_breakdown b
          ON o.order_id = b.order_id
          GROUP BY o.country
          ORDER BY total_sales DESC
          LIMIT 3
          """
top3 = sql.read_sql(query12, con = db_connection)
top3

country  total_sales
0          France   3108445.00
1         Germany   2411393.15
2  United Kingdom   2161854.80

#### 11.C Select Bottom 3 Country Based On Quantities

In [81]:
# Select Bottom 3 Country Based On Quantities

query13 = """
          SELECT o.country,
          SUM(b.quantity) as total_quantity
          FROM orders o
          JOIN orders_breakdown b
          ON o.order_id = b.order_id
          GROUP BY o.country
          ORDER BY total_quantity 
          LIMIT 3
          """
bottom3 = sql.read_sql(query13, con = db_connection)
bottom3

country  total_quantity
0  Finland             201
1  Denmark             204
2   Norway             261

#### 12. Select Average Sales By Categroy For The Country 'France'

In [82]:
# Select Average Sales By Categroy For The Country 'France'

query14 = """
          SELECT b.category,AVG(sales) AS avg_sales
          FROM orders o
          JOIN orders_breakdown b
          ON o.order_id = b.order_id
          WHERE o.country = 'France'
          GROUP BY b.category
          ORDER BY avg_sales DESC
          """
avg_sales = sql.read_sql(query14,con= db_connection)
avg_sales

category   avg_sales
0       Technology  595.145078
1        Furniture  582.314381
2  Office Supplies  167.217709

#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [109]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest

query15 = """
          SELECT o.country, b.category, AVG(b.sales*b.quantity -b.discount) AS avg_total_sales
          FROM orders o
          JOIN orders_breakdown b
          ON o.order_id = b.order_id
          GROUP BY o.country, b.category
          ORDER BY avg_total_sales DESC
          """

highest_total_sales = sql.read_sql(query15, con = db_connection)
highest_total_sales

country         category  avg_total_sales
0      Switzerland       Technology      6487.846154
1          Belgium       Technology      5000.526316
2           Norway        Furniture      4894.500000
3   United Kingdom       Technology      3798.169835
4            Spain       Technology      3676.299259
5          Germany       Technology      3198.119936
6           France        Furniture      3091.668395
7           France       Technology      2956.403756
8   United Kingdom        Furniture      2772.481111
9          Austria       Technology      2675.170213
10         Austria        Furniture      2629.400000
11         Finland        Furniture      2528.750000
12          Norway       Technology      2457.777778
13           Spain        Furniture      2365.364228
14         Germany        Furniture      2309.768750
15           Italy       Technology      2255.883422
16     Netherlands        Furniture      2209.955263
17           Italy        Furniture      2153.633824
18          Sweden        Furniture      2146.455556
19     Switzerland        Furniture      2054.545455
20        Portugal       Technology      1798.884615
21         Belgium        Furniture      1757.650000
22         Finland       Technology      1732.200000
23         Ireland       Technology      1605.738095
24     Netherlands       Technology      1536.406667
25        Portugal        Furniture      1520.300000
26         Ireland        Furniture      1198.225000
27        Portugal  Office Supplies      1191.606383
28         Denmark       Technology      1160.166667
29           Spain  Office Supplies      1100.528231
30         Belgium  Office Supplies      1038.187500
31         Austria  Office Supplies       908.389831
32          Sweden       Technology       897.705882
33          France  Office Supplies       847.168400
34  United Kingdom  Office Supplies       833.689574
35         Germany  Office Supplies       757.745446
36           Italy  Office Supplies       705.875000
37         Denmark        Furniture       674.555556
38     Switzerland  Office Supplies       642.944444
39         Finland  Office Supplies       637.342105
40          Norway  Office Supplies       571.108696
41          Sweden  Office Supplies       455.364662
42         Ireland  Office Supplies       411.559701
43     Netherlands  Office Supplies       398.090909
44         Denmark  Office Supplies       374.192308

### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



